In [ ]:
##### AIT 726 - Final Project Neural Network Code
##### By Japneet Kohli

####  The code is executed in the following order

1.   Load data.
2.   Select only translation writer and line translations data from the list of attributes.
3.   Preprocess text using regex.
5.   Use label encoder to assign numeric values to the categeorical labels of the translator writer.
6.   Split data into train and test.
7.   Toeknize words and pad sentences to convert words into vectors.
7.   Use GloVe pretrained word embeddings to create embedding matrix.
8.   Define bi-lstm  model with embedding layer, lstm layer and output layer.
9.   Compile and fit model.
10.  Predict values for test data and compute accuracy.


In [ ]:
# Load all the required packaegs and libraries.
import os
import re
import logging
import numpy as np
from numpy import zeros
import pandas as pd
import random
from nltk import word_tokenize
from nltk.util import ngrams
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, SimpleRNN, Activation,TimeDistributed,InputLayer,Bidirectional,LSTM,GRU
from keras import optimizers
from keras.optimizers import Adam
from keras import backend as K
import time
from keras.models import model_from_json
start_time = time.time()
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical

from gensim import models
print('Done')

Done


In [ ]:
#Mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load data
data=pd.read_csv("datadf.csv")

In [ ]:
# Display data
data.head()

,Unnamed: 0,writer,lines_translations_english_writer,lines_translations_english,lines_type
0,1,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...,Manglacharan
1,2,Guru Nanak Dev Ji,Bhai Manmohan Singh,"There is but one God. True is His Name, creati...",Manglacharan
2,5,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,Chant And Meditate:,Sirlekh
3,6,Guru Nanak Dev Ji,Bhai Manmohan Singh,Embrace His meditation.,Sirlekh
4,9,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...,Pankti


In [ ]:
# Check Shape
data.shape

(121020, 5)

In [ ]:
# List columns
data.columns

Index(['Unnamed: 0', 'writer', 'lines_translations_english_writer',
       'lines_translations_english', 'lines_type'],
      dtype='object')

In [ ]:
# Create subset of data with only required columns
columns_to_keep = ['writer','lines_translations_english_writer','lines_translations_english','lines_type']
newdata=data[columns_to_keep]

In [ ]:
# Display data
newdata.head()

,writer,lines_translations_english_writer,lines_translations_english,lines_type
0,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...,Manglacharan
1,Guru Nanak Dev Ji,Bhai Manmohan Singh,"There is but one God. True is His Name, creati...",Manglacharan
2,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,Chant And Meditate:,Sirlekh
3,Guru Nanak Dev Ji,Bhai Manmohan Singh,Embrace His meditation.,Sirlekh
4,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...,Pankti


In [ ]:
# Describe data
newdata.describe()

,writer,lines_translations_english_writer,lines_translations_english,lines_type
count,121020,121020,121020,121020
unique,35,2,112029,4
top,Guru Arjan Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. By The Grace Of The...,Pankti
freq,49746,60510,525,100400


In [ ]:
# Check Shape
newdata.shape

(121020, 4)

In [ ]:
# Check Info
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121020 entries, 0 to 121019
Data columns (total 4 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   writer                             121020 non-null  object
 1   lines_translations_english_writer  121020 non-null  object
 2   lines_translations_english         121020 non-null  object
 3   lines_type                         121020 non-null  object
dtypes: object(4)
memory usage: 3.7+ MB


In [ ]:
# Check Categorical Label Distribution
newdata.lines_translations_english_writer.value_counts()

Dr. Sant Singh Khalsa    60510
Bhai Manmohan Singh      60510
Name: lines_translations_english_writer, dtype: int64

In [ ]:
# Subset data to keep only columns for prediction
translator_columns=['lines_translations_english_writer','lines_translations_english']
translator_data=newdata[translator_columns]

In [ ]:
# Display data
translator_data.head()

,lines_translations_english_writer,lines_translations_english
0,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...
1,Bhai Manmohan Singh,"There is but one God. True is His Name, creati..."
2,Dr. Sant Singh Khalsa,Chant And Meditate:
3,Bhai Manmohan Singh,Embrace His meditation.
4,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...


In [ ]:
# Check Shape
translator_data.shape

(121020, 2)

In [ ]:
# Describe data
translator_data.describe()

,lines_translations_english_writer,lines_translations_english
count,121020,121020
unique,2,112029
top,Dr. Sant Singh Khalsa,One Universal Creator God. By The Grace Of The...
freq,60510,525


In [ ]:
# Check info
translator_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121020 entries, 0 to 121019
Data columns (total 2 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   lines_translations_english_writer  121020 non-null  object
 1   lines_translations_english         121020 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [ ]:
# Check if categorical distribution is the same as before
translator_data.lines_translations_english_writer.value_counts()

Dr. Sant Singh Khalsa    60510
Bhai Manmohan Singh      60510
Name: lines_translations_english_writer, dtype: int64

In [ ]:
# Check a random line of data
translator_data.lines_translations_english[14]

'Hundreds of thousands of clever tricks, but not even one of them will go along with you in the end.'

In [ ]:
# Check text of translated data
translator_data["lines_translations_english"]

0         One Universal Creator God. The Name Is Truth. ...
1         There is but one God. True is His Name, creati...
2                                       Chant And Meditate:
3                                   Embrace His meditation.
4         True In The Primal Beginning. True Throughout ...
                                ...                        
121015    These are the names of the sons of Maygh Raag....
121016    So all together, they sing the six Raagas and ...
121017    So all together, they sing the six Raagas and ...
121018    and all the forty-eight sons of the Raagas. ||...
121019    and all the forty-eight sons of the Raagas. ||...
Name: lines_translations_english, Length: 121020, dtype: object

In [ ]:
#Create function for data preprocessing using regular expressions
# Punctuation, HTML tags, non-ascii characters, etc, are all removed

def preprocess_text(txt):
        txt = re.sub(r'(?:<[^>]+>)', '', txt)#Html tags
        txt = re.sub('[^\x00-\x7F]+', ' ', txt) ##remove non ascii characters
        txt = re.sub('#\S+', '', txt)
        txt = re.sub('_', '', txt)
        txt = re.sub('-',' ',txt)
        txt = re.sub('/','',txt)
        txt = re.sub('~','',txt)
        txt = re.sub('|','',txt)
        txt = re.sub('\s+', ' ', txt) ## remove double spaces
        txt = re.sub(r'([A-Z][a-z]+)',lambda x:x.group().lower(),txt)##Lower case captilized words only but not words with all capitals
        return txt.strip()
data = [preprocess_text(t) for t in translator_data["lines_translations_english"]]

In [ ]:
# Check preprocessed data
data[0:12]

["one universal creator god. the name is truth. creative being personified. no fear. no hatred. image of the undying, beyond birth, self existent. by guru's grace",
 "there is but one god. true is his name, creative his personality and immortal his form. he is without fear sans enmity, unborn and self illumined. by the guru's grace he is obtained.",
 'chant and meditate:',
 'embrace his meditation.',
 'true in the primal beginning. true throughout the ages.',
 'true in the prime, true in the beginning of ages,',
 'true here and now. O nanak, forever and ever true. ||1||',
 'true he is even now and true he verily, shall be, O nanak!',
 'by thinking, he cannot be reduced to thought, even by thinking hundreds of thousands of times.',
 'by pondering on god, man cannot have a conception of him, even though he may ponder over lacs of times.',
 'by remaining silent, inner silence is not obtained, even by remaining lovingly absorbed deep within.',
 "even though one be silent and remains absorb

In [ ]:
# Fit data in array
data = np.array(data)

In [ ]:
# Check data
data[0:12]

array(["one universal creator god. the name is truth. creative being personified. no fear. no hatred. image of the undying, beyond birth, self existent. by guru's grace",
       "there is but one god. true is his name, creative his personality and immortal his form. he is without fear sans enmity, unborn and self illumined. by the guru's grace he is obtained.",
       'chant and meditate:', 'embrace his meditation.',
       'true in the primal beginning. true throughout the ages.',
       'true in the prime, true in the beginning of ages,',
       'true here and now. O nanak, forever and ever true. ||1||',
       'true he is even now and true he verily, shall be, O nanak!',
       'by thinking, he cannot be reduced to thought, even by thinking hundreds of thousands of times.',
       'by pondering on god, man cannot have a conception of him, even though he may ponder over lacs of times.',
       'by remaining silent, inner silence is not obtained, even by remaining lovingly absorbed de

In [ ]:
# Fit into dataframe
X = pd.DataFrame(data, columns = ["lines_translations_english"])

In [ ]:
X.head()

,lines_translations_english
0,one universal creator god. the name is truth. ...
1,"there is but one god. true is his name, creati..."
2,chant and meditate:
3,embrace his meditation.
4,true in the primal beginning. true throughout ...


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121020 entries, 0 to 121019
Data columns (total 1 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   lines_translations_english  121020 non-null  object
dtypes: object(1)
memory usage: 945.6+ KB


In [ ]:
X.lines_translations_english

0         one universal creator god. the name is truth. ...
1         there is but one god. true is his name, creati...
2                                       chant and meditate:
3                                   embrace his meditation.
4         true in the primal beginning. true throughout ...
                                ...                        
121015    these are the names of the sons of maygh raag....
121016    so all together, they sing the six raagas and ...
121017    so all together, they sing the six raagas and ...
121018    and all the forty eight sons of the raagas. ||...
121019    and all the forty eight sons of the raagas. ||...
Name: lines_translations_english, Length: 121020, dtype: object

In [ ]:
#Obtain labels by transforming categorical labels to numerical labels. 
X = X.lines_translations_english
Y = translator_data.lines_translations_english_writer
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1) 

In [ ]:
print(X.shape)
print(Y.shape)

(121020,)
(121020, 1)


In [ ]:
X[0:12]

0     one universal creator god. the name is truth. ...
1     there is but one god. true is his name, creati...
2                                   chant and meditate:
3                               embrace his meditation.
4     true in the primal beginning. true throughout ...
5     true in the prime, true in the beginning of ages,
6     true here and now. O nanak, forever and ever t...
7     true he is even now and true he verily, shall ...
8     by thinking, he cannot be reduced to thought, ...
9     by pondering on god, man cannot have a concept...
10    by remaining silent, inner silence is not obta...
11    even though one be silent and remains absorbed...
Name: lines_translations_english, dtype: object

In [ ]:
#Split data into train and test sets.
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, random_state=42)

In [ ]:
#Convert sentences into tokens and pad the sentences to make all sentences of uniform length.
from keras.preprocessing.text import Tokenizer
maxlen = len(max(X_train, key=len))
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train_matrix = sequence.pad_sequences(X_train,maxlen=maxlen)

In [ ]:
X_train_matrix.shape

(96816, 266)

In [ ]:
Y_train.shape

(96816, 1)

In [ ]:
print(maxlen)

266


In [ ]:
# Assign vocab size as 1+number of tokens
vocab_size = len(tokenizer.word_index) + 1
vocab_size

14295

In [ ]:
#Create function to create embedding matrix using GloVe pretrained embeddings for 6 billion words with 100 dimensions.

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix


In [ ]:
# Load embedding matrix
embedding_dim = 100
embedding_matrix = create_embedding_matrix(
    '/content/gdrive/My Drive/AIT 726 Final Project SGGS Ji/glove.6B.100d.txt',
     tokenizer.word_index, embedding_dim)

In [ ]:
embedding_matrix.shape

(14295, 100)

In [ ]:
# Create function to define bi-lstm rnn model
def RNN():
    inputs = Input(name='inputs',shape=[maxlen])
    layer = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=maxlen)(inputs)
    layer = Bidirectional(LSTM(100))(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.8)(layer)
    layer = Dense(1)(layer)
    layer = Activation('sigmoid')(layer)
    model1 = Model(inputs=inputs,outputs=layer)
    return model1

In [ ]:
# Compile model
model1 = RNN()
model1.summary()
model1.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.01),metrics=['accuracy'])

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 266)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 266, 100)          1429500   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_5 (Activation)    (None, 1)                

In [ ]:
# Fit model for training data
history = model1.fit(X_train_matrix,Y_train,batch_size=128,epochs=20,validation_split=0.05,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/20
719/719 [==============================] - 35s 49ms/step - loss: 0.1482 - accuracy: 0.9364 - val_loss: 0.2148 - val_accuracy: 0.9143
Epoch 2/20
719/719 [==============================] - 35s 49ms/step - loss: 0.1367 - accuracy: 0.9404 - val_loss: 0.2322 - val_accuracy: 0.9058


In [ ]:
# Preedictions for test data and compute accuracy
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=maxlen)

accr = model1.evaluate(test_sequences_matrix,Y_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1] * 100))

757/757 [==============================] - 8s 11ms/step - loss: 0.2151 - accuracy: 0.9175
Test set
  Loss: 0.215
  Accuracy: 91.745
